In [1]:
import xlearn as xl
import pandas as pd
import numpy as np
import gc
import time
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from pandas import DataFrame as DF
from keras.utils import to_categorical

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
data = pd.read_table('/home/ubuntu/data/UserMovie_train.txt',header=None,encoding='gb2312',delim_whitespace=True,index_col=None)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
"""
df = Dataframe to be converted to ffm format
Yype = train/test/val
Numerics=list of all numeric fields
Categories = list of all categorical fields
Features = list of all features except the Label and Id
"""
def convert_to_ffm(df,type,numerics,categories,features):
    currentcode = len(numerics)
    catdict={}
    catcodes={}
    #Flagging categorical and numerical fields
    for x in numerics:
        catdict[x]=0
    for x in categories:
        catdict[x]=1
    
    nrows = df.shape[0]
    ncolums=len(features)
    with open(str(type)+'_ffm.txt',"w") as text_file:
        #Looping over rows to convert each row to libffm format
        for n,r in enumerate(range(nrows)):
            datastring=""
            datarow=df.iloc[r].to_dict()
            datastring+=str(int(datarow['Label']))
            #For numerical fields,we are creating a dummy field here
            for i,x in enumerate(catdict.keys()):
                if(catdict[x]==0):
                    datastring=datastring+" "+str(i)+":"+str(i)+":"+str(datarow[x])
                else:
                    #For a new field appearing in a training example
                    if(x not in catcodes):
                        catcodes[x]={}
                        currentcode+=1
                        catcodes[x][datarow[x]]=currentcode #encdoing the feature
                    #For already encoded fields
                    elif(datarow[x] not in catcodes[x]):
                        currentcode+=1
                        catcodes[x][datarow[x]]=currentcode #encoding the feature
                    code = catcodes[x][datarow[x]]
                    datastring=datastring+" "+str(i)+":"+str(int(code))+":1"
                    
            datastring+='\n'
            text_file.write(datastring)


In [46]:
#数据预处理
result_list=list()
with open('/home/ubuntu/data/UserMovie_train.txt', 'r') as f:
    line = f.readline()
    result_list.append(line)
    line=f.readline()
    while line:
#         print(line)
        the_line=line.split('\t')
#         print(the_line)
        temp=''
        for i in range(len(the_line)):
            if i==2:
                temp+=the_line[i][0:4]+'\t'
            elif i==3:
                temp+=the_line[i][0]+'\n'
            else:
                temp+=the_line[i]+'\t'
#         print(temp)
        result_list.append(temp)
        line=f.readline()
#         break

In [47]:
with open('user_movie_processed.txt','w') as f:
    f.writelines(result_list)

In [51]:
readin = open('user_movie_processed.txt', 'r')
#write data file
output = open('libsvm.txt', 'w')
try:
    the_line = readin.readline()
    while the_line:
        # delete the \n
        the_line = the_line.strip('\n')
        output_line = ''
        temp=the_line.split('\t')
        for i in range(len(temp)):
            #the label col
            if i == 0:
                output_line = temp[len(temp)-1]
            #the features cols
            if temp[i-1] != 'NULL' and i != 0:
                the_text = ' ' + str(i) + ':' + temp[i-1]
                output_line = output_line + the_text
            i += 1
        output_line = output_line + '\n'
#         print(output_line)
#         break
        output.write(output_line)
        the_line = readin.readline()
finally:
    readin.close()

In [52]:
da2 = pd.read_table('libsvm.txt',header=None,encoding='gb2312',delim_whitespace=True,index_col=None)
da2.head()

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,0,1,2,3
0,star,1:uid,2:mid,3:timeStamp
1,5,1:1722994,2:1306505,3:2007
2,4,1:1405477,2:10574468,3:2013
3,5,1:15849871,2:4910186,3:2011
4,3,1:1068524,2:1304643,3:2006


In [ ]:
ffm_model = xl.create_ffm()
ffm_model.setTrain("criteo.tr.r100.gbdt0.ffm")
ffm_model.setValidate("criteo.va.r100.gbdt0.ffm")

param = {'task':'binary', # ‘binary’ for classification, ‘reg’ for Regression
         'k':2,           # Size of latent factor
         'lr':0.1,        # Learning rate for GD
         'lambda':0.0002, # L2 Regularization Parameter
         'Metric':'auc',  # Metric for monitoring validation set performance
         'epoch':25       # Maximum number of Epochs
        }

ffm_model.fit(param, "model.out")

In [4]:
data3 = pd.read_csv('/home/ubuntu//data/UserMovie_train.txt',sep='\t')

In [5]:
# uid 48613  mid 72541
mid = data3['mid'].unique()
uid = data3['uid'].unique()
collect_mid_encoder = {}
collect_mid_dencoder = {}
collect_uid_encoder = {}
collect_uid_dencoder = {}
for i in range(len(mid)):
    collect_mid_encoder[i] = mid[i]
    collect_mid_dencoder[mid[i]] = i
for i in range(len(uid)):
    collect_uid_encoder[i] = uid[i]
    collect_uid_dencoder[uid[i]] = i
data3['mid'] = data3['mid'].map(lambda x:collect_mid_dencoder[x])
data3['uid'] = data3['uid'].map(lambda x:collect_uid_dencoder[x])

In [6]:
# user_matrix = np.zeros((len(uid),len(mid)),dtype='float16')
# item_matrix = np.zeros((len(mid),len(uid)),dtype='float16')
data3.head(5)

,uid,mid,timeStamp,star
0,0,0,2007-08-22,5.0
1,1,1,2013-04-24,4.0
2,2,2,2011-12-20,5.0
3,3,3,2006-02-07,3.0
4,4,4,2008-09-22,4.0


In [7]:
def hash_star(x):
    if x>=3:
        return 1
    else:
        return 0

data3['star'] = data3['star'].apply(lambda x:hash_star(x))
# data3.groupby(['uid'])['star'].size()

In [8]:
data3['timeStamp'] = data3['timeStamp'].apply(lambda x:str(x)[0:4])

In [9]:
data3.head()

,uid,mid,timeStamp,star
0,0,0,2007,1
1,1,1,2013,1
2,2,2,2011,1
3,3,3,2006,1
4,4,4,2008,1


In [10]:
data3['timeStamp']=data3['timeStamp'].fillna('2006')

In [11]:
time_min=int(min(data3['timeStamp']))
time_max=int(max(data3['timeStamp']))

ValueError: invalid literal for int() with base 10: 'nan'

In [119]:
data3['timeStamp']=data3['timeStamp'].apply(lambda x:int(x)-time_min)

ValueError: invalid literal for int() with base 10: 'nan'